In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

alat = 5.107
ecutwfc = 20
ecutrho = 4 * ecutwfc

latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1], 
    [-1, -1,  1]
]).T

In [3]:
from qtmpy.lattice import RealLattice

reallat = RealLattice.from_alat(alat, *latvec_alat)
for i, ax in enumerate(reallat.axes_cart):
    print(f"a{i+1} : {ax}")

l_vec_cryst = np.eye(3)
l_vec_cart = reallat.cryst2cart(l_vec_cryst, axis=1)

print(l_vec_cart)

l_vec_cart = reallat.latvec
l_vec_cryst = reallat.cart2cryst(l_vec_cart, axis=0)
print(l_vec_cryst)

l_vec_cryst = reallat.alat2cryst(l_vec_cart / reallat.alat, axis=0)
print(l_vec_cryst)

a1 : [2.5535, -2.5535, -2.5535]
a2 : [2.5535, 2.5535, -2.5535]
a3 : [2.5535, 2.5535, 2.5535]
[[ 2.5535 -2.5535 -2.5535]
 [ 2.5535  2.5535 -2.5535]
 [ 2.5535  2.5535  2.5535]]
[[ 1.00000000e+00 -5.55111512e-17 -5.55111512e-17]
 [ 0.00000000e+00  1.00000000e+00  1.66533454e-16]
 [-5.55111512e-17  5.55111512e-17  1.00000000e+00]]
[[ 1.00000000e+00 -7.08738623e-17 -7.08738623e-17]
 [ 0.00000000e+00  1.00000000e+00  1.41747725e-16]
 [-7.08738623e-17  7.08738623e-17  1.00000000e+00]]


In [4]:
from qtmpy.config import qtmconfig
from qtmpy.lattice import ReciLattice
from qtmpy.gspace.gspc import GSpace

qtmconfig.fft_backend = 'numpy'

recilat = ReciLattice.from_reallat(reallat)
gspc = GSpace(recilat, ecutrho)

In [5]:
import h5py

f = h5py.File('./charge-density.hdf5')

print(f.keys())
out = np.array(f['MillerIndices'])
print(np.all(out == gspc.g_cryst.T[gspc.idxsort]))
qe2qtm_map = np.argsort(gspc.idxsort)
print(np.all(out[qe2qtm_map] == gspc.g_cryst.T))
print(qe2qtm_map)
print(gspc.g_cryst.T)
rhotot_g = np.array(f['rhotot_g']).view('c16')
rhodiff_g = np.array(f['rhodiff_g']).view('c16')
print(rhotot_g.shape)
f.close()

<KeysViewHDF5 ['MillerIndices', 'rhodiff_g', 'rhotot_g']>
False
False
[  0   1  43 ... 200  78  18]
[[ 0  0  0]
 [ 0  0  1]
 [ 0  0  2]
 ...
 [-1 -1 -3]
 [-1 -1 -2]
 [-1 -1 -1]]
(2243,)


In [6]:
print(np.amax(np.abs(rhotot_g.imag)))
print(np.amax(np.abs(rhotot_g.real)))
print(rhotot_g[0])

8.757842488417447e-19
0.24024391957368915
(0.24024391957368915+0j)


In [7]:
from qtmpy.field import GField

rho = GField(gspc, np.array([rhotot_g, rhodiff_g]))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (base.py, line 17)

In [ ]:
from qtmpy.field import concatenate, stack

out = stack((rho[0], rho[1]), axis=0)

In [ ]:
rho[0] = (rho[0] + rho[1]) / 2

In [9]:
import cupy as cp

reallat = RealLattice(alat, alat * cp.array(latvec_alat).T)
for i, ax in enumerate(reallat.axes_cart):
    print(f"a{i+1} : {ax}")

l_vec_cryst = cp.eye(3)
l_vec_cart = reallat.cryst2cart(l_vec_cryst, axis=1)

print(l_vec_cart)

l_vec_cart = reallat.latvec
l_vec_cryst = reallat.cart2cryst(l_vec_cart, axis=0)
print(l_vec_cryst)

l_vec_cryst = reallat.alat2cryst(l_vec_cart / reallat.alat, axis=0)
print(l_vec_cryst)

a1 : [2.5535, -2.5535, -2.5535]
a2 : [2.5535, 2.5535, -2.5535]
a3 : [2.5535, 2.5535, 2.5535]
[[ 2.5535 -2.5535 -2.5535]
 [ 2.5535  2.5535 -2.5535]
 [ 2.5535  2.5535  2.5535]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-5.55111512e-17 -5.55111512e-17  1.00000000e+00]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-7.08738623e-17 -7.08738623e-17  1.00000000e+00]]


In [10]:
qtmconfig.use_gpu = True

In [11]:
gspc_gpu = gspc.dev

In [12]:
print(gspc_gpu.g_cryst)

[[ 0  0  0 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]
 [ 0  1  2 ... -3 -2 -1]]


In [23]:
from types import MethodType


class Test:
    
    def __init__(self, flag):
        self.flag = flag
        self.attr1 = 10
    
    @property
    def attr2(self):
        return 100
        
    def func1(self):
        print('hello world')
    
    def func2(self, arg1):
        print(arg1)
        
    def __enter__(self):
        print('entering')
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exiting')
        
    def __getattribute__(self, item):
        # print(f'getattribute: {item}')
        out = super().__getattribute__(item)
        if isinstance(out, MethodType) and not self.flag:
            raise ValueError("disabled")
        return out

In [24]:
a = Test(True)
print(a.attr1, a.attr2)
a.func1()
a.func2('abc')

with a:
    print('here')


b = Test(False)

with b:
    print('here')
    
print(b.attr1)
print(b.attr2)
b.func1()
b.func2(a)

10 100
hello world
abc
entering
here
exiting
entering
here
exiting
10
100


ValueError: disabled

In [4]:
from mpi4py.MPI import Op, SUM, PROD
from typing import Union

t = Union['Op', str]
print(isinstance('SUM', t))

TypeError: issubclass() arg 2 must be a class, a tuple of classes, or a union